In [2]:
import nltk
import numpy as np
import pandas as pd
# from nltk.corpus import twitter_samples 

from utils import process_tweet, build_freqs

https://www.kaggle.com/mustfkeskin/turkish-movie-sentiment-analysis-dataset

sitesinden indirdiğim verileri sanki tweet'miş gibi kullanıyorum.

In [4]:
movie_reviews = pd.read_csv("turkish_movie_sentiment_dataset.csv")
all_positive_tweets = list()
all_negative_tweets = list()
for i in range(movie_reviews.shape[0]):
    if(float(movie_reviews['point'][i].replace(',','.')) >= 2.5):
        all_positive_tweets.append(movie_reviews['comment'][i])
    else:
        all_negative_tweets.append(movie_reviews['comment'][i])
all_positive_tweets = all_positive_tweets[0:5000]
all_negative_tweets = all_negative_tweets[0:5000]

In [5]:
# split the data into two pieces, one for training and one for testing (validation set) 
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

In [6]:
# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [7]:
# Print the shape train and test sets
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


In [8]:
# create frequency dictionary
freqs = build_freqs(train_x, train_y)
# print(freqs)
# print(freqs.get(('sleeep', 1.0)) if freqs.get(('sleeep', 1.0)) != None else 0)
# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs)))

type(freqs) = <class 'dict'>
len(freqs) = 43270


In [9]:
# test the function below
print('This is an example of a positive tweet: \n', train_x[0])
print('\nThis is an example of the processed version of the tweet: \n', process_tweet(train_x[0]))

This is an example of a positive tweet: 
 
                      Jean Reno denince zaten leon filmi gelir akla izlemeyen kalmamıştır ama kaldıysada ee ne duruyorsun hemen izle :)
        
            

This is an example of the processed version of the tweet: 
 ['jean', 'reno', 'den', 'zate', 'leon', 'filmi', 'gelir', 'ak', 'izlemeye', 'kalmamış', 'kaldıysa', 'ee', 'duruyor', 'heme', 'iz', ':)']


In [10]:
# UNQ_C1 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def sigmoid(z): 
    '''
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    '''
    
    # calculate the sigmoid of z
    h = 1/(1 + np.exp(-z))
    
    return h

In [11]:
# UNQ_C2 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    Hint: you might want to print the cost to make sure that it is going down.
    '''
    # get 'm', the number of rows in matrix x
    m = x.shape[0]
    
    for i in range(0, num_iters):
        
        # get z, the dot product of x and theta
        z = np.dot(x, theta)
        
        # get the sigmoid of z
        h = sigmoid(z)
        
        # calculate the cost function
        J = (-1/m) *np.sum(np.dot(np.transpose(y), np.log(h)) + np.dot(np.transpose(1 - y), np.log(1 - h)))

        # update the weights theta
        theta = theta - (alpha/m) * np.dot(np.transpose(x), h - y)
        
    J = float(J)
    return J, theta

In [12]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def extract_features(tweet, freqs):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1 
    
    
    # loop through each word in the list of words
    for word in word_l:
        
        # increment the word count for the positive label 1
        x[0,1] += freqs.get((word, 1.0)) if freqs.get((word, 1.0)) != None else 0
        
        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word, 0.0)) if freqs.get((word, 0.0)) != None else 0
        
    assert(x.shape == (1, 3))
    return x

In [13]:
tmp2 = extract_features('blorb bleeeeb bloooob', freqs)
print(tmp2)

[[1. 0. 0.]]


In [14]:
# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

# Apply gradient descent
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

The cost after training is 0.65154724.
The resulting vector of weights is [0.0, 0.0001749, -0.00017639]


In [15]:
# UNQ_C4 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def predict_tweet(tweet, freqs, theta):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    
    # extract the features of the tweet and store it into x
    x = extract_features(tweet, freqs)
    
    # make the prediction using x and theta
    y_pred = sigmoid(np.dot(x, theta))
        
    return y_pred

In [16]:
# Run this cell to test your function
for tweet in ['güzel filmmiş beğendim', 'oyunculuklar baya kötü, sakın izlemeyin, zaman kaybı', '5 defa izledim']:
    print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))

güzel filmmiş beğendim -> 0.528802
oyunculuklar baya kötü, sakın izlemeyin, zaman kaybı -> 0.455940
5 defa izledim -> 0.498692


In [17]:
# UNQ_C5 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def test_logistic_regression(test_x, test_y, freqs, theta):
    """
    Input: 
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output: 
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """
    # the list for storing predictions
    y_hat = []
    
    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)
        
        if y_pred > 0.5:
            # append 1.0 to the list
            y_hat.append(1.0)
        else:
            # append 0 to the list
            y_hat.append(0)

    # With the above implementation, y_hat is a list, but test_y is (m,1) array
    # convert both to one-dimensional arrays in order to compare them using the '==' operator
    m = test_y.shape[0]
    accuracy = np.sum(test_y.reshape(m) == np.array(y_hat))/m

    
    return accuracy

In [18]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Logistic regression model's accuracy = 0.6855


In [21]:
# Feel free to change the tweet below
my_tweet = 'oyunculuklar çok kötü, senaryo desen vasat'
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

['oyunculuk', 'köt', 'senaryo', 'dese', 'vasat']
[[0.45819065]]
Negative sentiment


In [22]:
# Feel free to change the tweet below
my_tweet = 'tam bir başyapıt, ne zaman canım sıkılsa açıp bu filmi izlerim '
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

['tam', 'bir', 'başyapıt', 'zama', 'ca', 'sıkıl', 'açıp', 'filmi', 'iz']
[[0.50909632]]
Positive sentiment
